In [2]:
import numpy as np
from scipy.sparse import diags
%run init_function.ipynb
qn_set = select_qn()
r, u_funcs = radial_function_set(qn_set)
def Coulomb(r, Z=6):
    """
    计算库伦势能矩阵（对角矩阵）
    r: 1D 径向网格数组，长度 n;
    Z: 原子序数
    返回: scipy.sparse.csr_matrix 形状 (n, n)
    """
    eps = 1e-12  # 避免 r=0 除零
    diag = -Z / np.maximum(r, eps)
    Vc = diags(diag, offsets=0, format="csr")
    return Vc

def Hartree(r,u_funcs):
    """
    生成 RHF 的 Hartree 势（球对称）。

    参数：
    r: 1D 径向网格，升序，len = n;
    u_funcs: 形如 (m, n) 的数组或列表，每行/列是一条占据轨道在 r 上的取值
    
    返回:
    V_H(r) 一维数组，长度 n
    """
    r = np.asarray(r, dtype=np.float64)
    rho = np.sum(np.abs(u_funcs)**2, axis=0).astype(np.float64)  # ρ(r) = Σ |u|^2
    dr = np.diff(r)
    if not np.all(dr > 0):
        raise ValueError("r 必须严格递增")

    # 左积累: ∫_0^{r_i} ρ(r') r'^2 dr'
    left = np.zeros_like(rho)
    left[1:] = np.cumsum(0.5 * (rho[1:] * r[1:]**2 + rho[:-1] * r[:-1]**2) * dr)

    # 右积累: ∫_{r_i}^∞ ρ(r') r' dr'
    right = np.zeros_like(rho)
    right[-1] = 0.0
    right[-2::-1] = np.cumsum(0.5 * (rho[-1:0:-1] * r[-1:0:-1] + rho[-2::-1] * r[-2::-1]) * dr[::-1])

    eps = 1e-12
    V_H = 4 * np.pi * (left / np.maximum(r, eps) + right)
    V_H_matrix = diags(V_H, offsets=0, format="csr")
    return V_H_matrix
np.shape(Hartree(r,u_funcs))


<>:8: SyntaxWarning: invalid escape sequence '\p'
C:\Users\win\AppData\Local\Temp\ipykernel_34348\1189739581.py:8: SyntaxWarning: invalid escape sequence '\p'
  """
C:\Users\win\AppData\Local\Temp\ipykernel_34348\1189739581.py:26: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  normalization = np.sqrt((2 * Z / n) * (np.math.factorial(n - l - 1) / (2 * n * np.math.factorial(n + l))))


(1000, 1000)